In [1]:
#Snowpark lib
#from snowflake.snowpark import Session

# Data Science Libs
import numpy as np
import pandas as pd

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import DBSCAN
import pickle
import requests

np.random.seed(0)

In [2]:
# !pip install scikit-learn==1.3.2

In [3]:
# from fosforml.model_manager.snowflakesession import get_session
# my_session = get_session()

In [4]:
with open('Churn_Classifier.pkl','rb') as f:  ## use rb while reading the fie
    churn_model = pickle.load(f)

In [5]:
churn_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['FUND_TOTAL_ASSETS',
                                                   'FUND_RETURN_TARGET_PERCENTAGE',
                                                   'CASH_BENCHMARK_ALLOCATION',
                                                   'FIXED_INCOME_BENCHMARK_ALLOCATION',
                                                   'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION',
                                                   'INTERNATIONAL_LISTED_EQU...
                                                                                    score_func=<function chi2 at 0x7ff42cd32dc0>))]),
                                                  ['MEMBER_GENDER',
                                                   'MEMBER_STATE',
                                                   'MEMBER_CONTACT_VERIFIED',
                                                   'INVESTMENT_RISK_CATEGORY',
                                                   'FUND_RISK_LEVEL',
                                                   'FUND_RISK_CATEGORY',
                                                   'NEG_NETRETURN_SINCE_INCEPTION',
                                                   'INVESTMENT_AGE_GROUP',
                                                   'RETIREMENT_AGE_GROUP',
                                                   'TOTAL_FUNDS_INVESTED'])])),
                ('classifier',
                 RandomForestClassifier(max_depth=8, max_features=0.6,
                                        max_samples=0.8, n_estimators=500))])

In [6]:
import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import col
from snowflake.snowpark import Session


connection_parameters = {
 "account": "fya62509",
 "region":"us-east-1",
 "user": "MANISHH",
 "password": "cXl2n5i6#",
 "role": "MANISHH",  # optional
 "warehouse": "FOSFOR_SOLUTIONS_WH",  # optional
 "database": "FDC_BANKING_FS",  # optional
 "schema": "BFS_SUPER_ANNU_SCHEMA",  # optional 
}
my_session = Session.builder.configs(connection_parameters).create()  

In [7]:
table_name = 'MEMBER_FUNDS_ENRICHED_DETAILS'
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [8]:
df = df.drop(["MEMBER_DOB_DT", 'ALLOCATION_DT', "CHURN_DT", "RETIREMENT_DT"], axis =1 )

In [9]:
frame = df[['MEMBER_GENDER', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED','FUND_TOTAL_ASSETS','FUND_RETURN_TARGET_PERCENTAGE',
            'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION','FIXED_INCOME_BENCHMARK_ALLOCATION',
            'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
            'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION',
            'INFRA_BENCHMARK_ALLOCATION','COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION','FUND_RISK_LEVEL', 
            'FUND_RISK_CATEGORY','NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS','YEAR_5_RETURNS',
            'YEAR_7_RETURNS','YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',
            'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', 'TOTAL_FUNDS_INVESTED','CHURN_FLAG']].copy()

X = frame.drop("CHURN_FLAG", axis=1)
y = frame["CHURN_FLAG"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [10]:
def score(model, request):
    payload_dict = eval(request.json["payload"])
    data = pd.DataFrame.from_dict(payload_dict)
    frame = data[['MEMBER_GENDER', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED','FUND_TOTAL_ASSETS','FUND_RETURN_TARGET_PERCENTAGE',
            'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION','FIXED_INCOME_BENCHMARK_ALLOCATION',
            'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
            'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION',
            'INFRA_BENCHMARK_ALLOCATION','COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION','FUND_RISK_LEVEL', 
            'FUND_RISK_CATEGORY','NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS','YEAR_5_RETURNS',
            'YEAR_7_RETURNS','YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',
            'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', 'TOTAL_FUNDS_INVESTED']].copy()
    result = model.predict(frame)
    prediction = pd.DataFrame({"CHURN_PREDICTION":result})
    prediction["CHURN_PREDICTION"] = prediction["CHURN_PREDICTION"].apply(lambda x: "Y" if x == 1 else "N")
    probability = model.predict_proba(frame)
    prediction["Probability"] = [round(k[1],4) for k in probability]    
    prediction = prediction.to_dict()
    return pd.DataFrame(prediction)

In [11]:
table_name = 'MEMBER_FUNDS_ENRICHED_DETAILS'
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()
df = df.drop(["MEMBER_DOB_DT", 'ALLOCATION_DT', "CHURN_DT", "RETIREMENT_DT"], axis =1 )

In [12]:
payload = df.head(10).to_dict()
req = requests.Request()
req.json = {"payload":str(payload)}
y = req
yo = score(churn_model, y)
yo

,CHURN_PREDICTION,Probability
0,Y,0.9681
1,Y,0.9057
2,Y,0.7971
3,Y,0.7901
4,Y,0.8173
5,Y,0.8144
6,Y,0.5759
7,Y,0.9588
8,Y,0.9940
9,Y,0.9118


In [13]:
type(yo)

pandas.core.frame.DataFrame

In [14]:
frame = df[['MEMBER_GENDER', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED','FUND_TOTAL_ASSETS','FUND_RETURN_TARGET_PERCENTAGE',
            'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION','FIXED_INCOME_BENCHMARK_ALLOCATION',
            'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
            'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION',
            'INFRA_BENCHMARK_ALLOCATION','COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION','FUND_RISK_LEVEL', 
            'FUND_RISK_CATEGORY','NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS','YEAR_5_RETURNS',
            'YEAR_7_RETURNS','YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',
            'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', 'TOTAL_FUNDS_INVESTED','CHURN_FLAG']].copy()

X = frame.drop("CHURN_FLAG", axis=1)
y = frame["CHURN_FLAG"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [15]:
payload = X_test.to_dict()
req = requests.Request()
req.json = {"payload":str(payload)}
y = req
pred = score(churn_model, y)

In [16]:
#temp_ = eval(pred_dic)
#print("string to dict")
#pred = pd.DataFrame.from_dict(temp_)

In [17]:
y_pred = pred["CHURN_PREDICTION"]
y_prob = pred["Probability"]

In [18]:
from fosforml import register_model
## registering the model in Fosfor Insight Designer.

In [19]:
churn_model.feature_names_in_

array(['MEMBER_GENDER', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED',
       'FUND_TOTAL_ASSETS', 'FUND_RETURN_TARGET_PERCENTAGE',
       'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION',
       'FIXED_INCOME_BENCHMARK_ALLOCATION',
       'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION',
       'INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
       'UNLISTED_EQUITY_BENCHMARK_ALLOCATION',
       'EQUITY_BENCHMARK_ALLOCATION', 'PROPERTY_BENCHMARK_ALLOCATION',
       'INFRA_BENCHMARK_ALLOCATION', 'COMMODITIES_BENCHMARK_ALLOCATION',
       'OTHERS_BENCHMARK_ALLOCATION', 'FUND_RISK_LEVEL',
       'FUND_RISK_CATEGORY', 'NEG_NETRETURN_SINCE_INCEPTION',
       'YEAR_1_RETURNS', 'YEAR_3_RETURNS', 'YEAR_5_RETURNS',
       'YEAR_7_RETURNS', 'YEAR_10_RETURNS', 'SUPER_FEES', 'PENSION_FEES',
       'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP',
       'TOTAL_FUNDS_INVESTED'], dtype=object)

In [20]:
y_train = y_train.to_frame()
y_test = y_test.to_frame()
y_pred = y_pred.to_frame()
y_prob = y_prob.to_frame()

In [21]:
type(X_train), type(X_test),type(y_train), type(y_test), type(y_pred)

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame)

In [25]:
register_model(
    model_obj=churn_model, 
    session=my_session,
    x_train=X_train,
    y_train=y_train,
    x_test=X_test,
    y_test=y_test,
    y_pred=y_pred,
    y_prob = y_prob,
    source="Notebook",
    dataset_name="MEMBER_FUNDS_ENRICHED_DETAILS",
    dataset_source="Snowflake",
    #dataset_source="InMemory",
    name="CHURN_BINARY_MODEL",
    description="PREDICTING_FUND_CHURN",
    flavour="sklearn",
    model_type="classification",
    conda_dependencies=["scikit-learn==1.3.2"]
)

# register_model(
#   model_obj=churn_model,
#   session=my_session,
#   x_train=X_train,
#   y_train=y_train,
#   x_test=X_test,
#   y_test=y_test,
#   y_pred=y_pred,
#   y_prob=y_prob,
#   source="NOTEBOOK",
#   dataset_name="MEMBER_FUNDS_ENRICHED_DETAILS",
#   dataset_source="InMemory",
#   name="CHURN_BINARY_MODEL",
#   description="PREDICTING_FUND_CHURN",
#   flavour="sklearn",
#   model_type="classification",
#   conda_dependencies=["scikit-learn==1.3.2"]
# )

       Probability
0           0.9325
1           0.0000
2           0.0000
3           0.9136
4           0.0000
...            ...
67195       0.1889
67196       0.5841
67197       0.7707
67198       0.0000
67199       0.0000

[67200 rows x 1 columns]
> /notebooks/notebooks/fosforml/model_manager/model_registry.py(54)register_model()
     52         import pdb
     53         pdb.set_trace()
---> 54         metrics_errors,metrics = self.get_model_metrics(
     55                                         source=source,
     56                                         model=model,



ipdb>  c


> /notebooks/notebooks/fosforml/model_manager/model_registry.py(543)get_dataset_details()
    541                     import pdb
    542                     pdb.set_trace()
--> 543                     model_artifacts = load_model_artifacts(
    544                                                         session=session,
    545                                                         model_type=model_type,



ipdb>  c


> /notebooks/notebooks/fosforml/utils.py(199)load_model_artifacts()
    197     import pdb
    198     pdb.set_trace()
--> 199     from fosforml.model_manager import ModelObject
    200     model_instance = ModelObject(model_obj,
    201                                 model_type,



...

ipdb>  c


Calculating build time metrics

Progress: ██████████████                                                         20.0%
Calculating build time metrics

Progress: ████████████████████████████                                           40.0%
(0000) pos_label=1 is not a valid label. It must be one of ['N', 'Y']
(0000) pos_label=1 is not a valid label. It must be one of ['N', 'Y']
(0000) pos_label=1 is not a valid label. It must be one of ['N', 'Y']


(1300) (1304): 01b6c20f-070d-e96d-0072-f303106beca6: 100038 (22018): Numeric value 'Y' is not recognized
Calculating build time metrics

Progress: ██████████████████████████████████████████                             60.0%


Error in while calculating roc_auc 
Calculating build time metrics

Progress: ████████████████████████████████████████████████████████               80.0%
Error in while calculating feature_importance 
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%
model dataset upload failed, error: (2101) Dataset INSIGHT_DESIGNER_SPCS.INSIGHT_SPCS_SCHEMA.MODEL_738E8045_9A80_436D_B538_2A8EED5DF879_FDC_CHURN_BINARY_MODEL_v3 does not exist or is inaccessible.
Failed to set tags for model MODEL_738E8045_9A80_436D_B538_2A8EED5DF879_FDC_CHURN_BINARY_MODEL.
(1300) (1304): 01b6c20f-070d-ea00-0072-f303106c00d6: 003001 (42501): SQL access control error:
Insufficient privileges to operate on tag 'FLAVOR'


"Model 'MODEL_738E8045_9A80_436D_B538_2A8EED5DF879_FDC_CHURN_BINARY_MODEL' registered successfully."

In [22]:
y_prob

,Probability
0,0.9325
1,0.0000
2,0.0000
3,0.9136
4,0.0000
...,...
67195,0.1889
67196,0.5841
67197,0.7707
67198,0.0000


In [ ]:
model_reg = register_model(churn_model,
                           score, 
                           name="CHURN_BINARY_MODEL", 
               description="PREDICTING_FUND_CHURN",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="\\n pip install --q fosforml==1.0.1b1",
               y_true=y_test,
               y_pred=y_pred,
               prob=y_prob,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train,
               y_test=y_test,
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)

# CHURN REASON MODEL

In [ ]:
with open('reason_model.pkl','rb') as f:  ## use rb while reading the fie
    reason_model = pickle.load(f)

In [ ]:
churned = df[df["CHURN_FLAG"] == "Y"]

In [ ]:
r2id = {'NO REASON IDENTIFIED': 0,
 'HIGH ACCOUNT FEE': 1,
 'POOR FUND PERFORMANCE': 2,
 'HIGH TRANSACTION FEE': 3,
 'FUND REPUTATION DECLINING': 4,
 'LIFE EVENT': 5}
id2r = {v:k for k, v in r2id.items()}
id2r

In [ ]:
@scoring_func
def score(model, request):
    payload_dict = eval(request.json["payload"])
    data = pd.DataFrame.from_dict(payload_dict)
    frame = data[['MEMBER_GENDER', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED','FUND_TOTAL_ASSETS','FUND_RETURN_TARGET_PERCENTAGE',
            'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION','FIXED_INCOME_BENCHMARK_ALLOCATION',
            'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
            'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION',
            'INFRA_BENCHMARK_ALLOCATION','COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION','FUND_RISK_LEVEL', 
            'FUND_RISK_CATEGORY','NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS','YEAR_5_RETURNS',
            'YEAR_7_RETURNS','YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',
            'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', 'TOTAL_FUNDS_INVESTED']].copy()
    id2r = {0: 'NO REASON IDENTIFIED',
             1: 'HIGH ACCOUNT FEE',
             2: 'POOR FUND PERFORMANCE',
             3: 'HIGH TRANSACTION FEE',
             4: 'FUND REPUTATION DECLINING',
             5: 'LIFE EVENT'}
    result = model.predict(frame)
    prediction = pd.DataFrame({"REASON_PREDICTION":result})
    prediction["REASON_PREDICTION"] = prediction["REASON_PREDICTION"].apply(lambda x: id2r[x])
    prediction = str(prediction.to_dict())
    return prediction

In [ ]:
payload = churned.head(10).to_dict()
req = requests.Request()
req.json = {"payload":str(payload)}
y = req
yo = score(reason_model, y)
yo

In [ ]:

X = churned.drop("CHURN_REASON", axis=1)
y = churned["CHURN_REASON"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
payload = X_test.to_dict()
req = requests.Request()
req.json = {"payload":str(payload)}
y = req
pred_dic = score(reason_model, y)

In [ ]:
temp_ = eval(pred_dic)
print("string to dict")
pred = pd.DataFrame.from_dict(temp_)

In [ ]:
y_pred = pred["REASON_PREDICTION"]
# y_prob = pred["Probability"]

In [ ]:
y_prob = reason_model.predict_proba(X_test)

In [ ]:
y_test

In [ ]:
y_pred

In [ ]:
## registering the model in Fosfor.
model_reg = register_model(reason_model,
               score, 
               name="CHURN_REASON_MODEL", 
               description="PREDICTING_FUND_CHURN_REASON",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="\\n pip install --q fosforml==1.0.1b1",
               y_true=y_test,
               y_pred=y_pred,
               prob=y_prob,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train,
               y_test=y_test,
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)